In [2]:
from vistiq.core import StackProcessor, StackProcessorConfig
from vistiq.preprocess import DoGConfig, DoG
from vistiq.seg import OtsuThresholdConfig, OtsuThreshold, LocalThresholdConfig, LocalThreshold
from vistiq.seg import LabellerConfig, Labeller
from vistiq.seg import RegionAnalyzerConfig, RegionAnalyzer, RegionFilterConfig, RegionFilter, RangeFilterConfig, RangeFilter
from vistiq.seg import SegmenterConfig, Segmenter
from vistiq.utils import ArrayIteratorConfig, ArrayIterator
from vistiq.analysis import CoincidenceDetectorConfig, CoincidenceDetector

import numpy as np
import stackview

In [3]:
path = "~/Documents/SDS_/projects/Siegrist/Microsam_Segmentation"

In [4]:
import ipywidgets as widgets
from pathlib import Path
import os
from bioio import BioImage
from vistiq.utils import load_image

# Expand the path from cell 1
base_path = Path(path).expanduser()

# Common image file extensions
IMAGE_EXTENSIONS = {'.tif', '.tiff', '.png', '.jpg', '.jpeg', '.bmp', '.gif', '.nd2', 
                    '.czi', '.lif', '.ome.tif', '.ome.tiff', '.zarr', '.svs', '.vsi'}

# Recursively get list of all image files in the directory and subdirectories
file_list = []
if base_path.exists() and base_path.is_dir():
    # Use rglob to recursively search for all files
    all_files = sorted(base_path.rglob('*'))
    # Filter to only image files (not directories) and get relative paths
    file_list = [str(f.relative_to(base_path)) for f in all_files 
                 if f.is_file() and f.suffix.lower() in IMAGE_EXTENSIONS]
    if not file_list:
        file_list = ["No image files found"]
else:
    file_list = ["Directory not found"]

# Create file dropdown widget
file_dropdown = widgets.Dropdown(
    options=file_list,
    value=file_list[0] if file_list else None,
    description='File:',
    disabled=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

# Create scene dropdown widget (will be updated when file is selected)
scene_dropdown = widgets.Dropdown(
    options=[],
    description='Scene:',
    disabled=True,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

# Create channel selector widget (will be updated when file or scene is selected)
# Using SelectMultiple to allow selection of multiple channels
channel_dropdown = widgets.SelectMultiple(
    options=[],
    description='Channels:',
    disabled=True,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px', height='150px')
)

# Initialize img variable in global namespace
img = None
scale = None



In [5]:
def update_channels(file_path_str, scene_index=None):
    """Update channel dropdown based on file and scene selection."""
    # Disable channel dropdown while loading
    channel_dropdown.disabled = True
    channel_dropdown.options = ["Loading..."]
    
    try:
        # Use bioio to get available channels
        reader = BioImage(str(file_path_str))
        
        # Set scene if provided
        if scene_index is not None:
            reader.set_scene(scene_index)
        
        # Try to get channel information
        # Check if reader has channel information
        if hasattr(reader, 'channel_names') and reader.channel_names:
            channel_names = reader.channel_names
            if isinstance(channel_names, (list, tuple)):
                channel_options = [f"Channel {i}: {name}" if name else f"Channel {i}" 
                                 for i, name in enumerate(channel_names)]
            else:
                num_channels = len(channel_names) if hasattr(channel_names, '__len__') else 1
                channel_options = [f"Channel {i}" for i in range(num_channels)]
        elif hasattr(reader, 'dims') and 'C' in reader.dims:
            # Try to get channel dimension from xarray
            try:
                xdata = reader.xarray_dask_data
                if hasattr(xdata, 'dims') and 'C' in xdata.dims:
                    num_channels = xdata.dims['C']
                    channel_options = [f"Channel {i}" for i in range(num_channels)]
                else:
                    channel_options = ["Channel 0 (only channel)"]
            except:
                channel_options = ["Channel 0 (only channel)"]
        else:
            # Try to get shape and infer channels
            try:
                test_data = reader.get_image_data()
                if test_data is not None and hasattr(test_data, 'shape'):
                    # Check if last dimension might be channels (typically for RGB)
                    if test_data.ndim >= 3 and test_data.shape[-1] in [1, 3, 4]:
                        num_channels = test_data.shape[-1]
                        channel_options = [f"Channel {i}" for i in range(num_channels)]
                    else:
                        channel_options = ["Channel 0 (only channel)"]
                else:
                    channel_options = ["Channel 0 (only channel)"]
            except:
                channel_options = ["Channel 0 (only channel)"]
        
        channel_dropdown.options = channel_options
        # For SelectMultiple, value should be a tuple/list, default to first channel if available
        channel_dropdown.value = (channel_options[0],) if channel_options else ()
        channel_dropdown.disabled = False
        
    except Exception as e:
        channel_dropdown.options = [f"Error: {str(e)}"]
        channel_dropdown.disabled = True


In [6]:
def update_scenes(change):
    """Update scene dropdown when file selection changes."""
    selected_relative_path = change['new']
    
    # Check if we have a valid selection
    if not selected_relative_path or selected_relative_path in ["Directory not found", "No image files found"]:
        scene_dropdown.options = []
        scene_dropdown.disabled = True
        channel_dropdown.options = []
        channel_dropdown.disabled = True
        return
    
    if not base_path.exists() or not base_path.is_dir():
        scene_dropdown.options = ["Directory not found"]
        scene_dropdown.disabled = True
        channel_dropdown.options = []
        channel_dropdown.disabled = True
        return
    
    # Construct full path by joining base_path with relative path
    file_path = base_path / selected_relative_path
    
    # Disable scene dropdown while loading
    scene_dropdown.disabled = True
    scene_dropdown.options = ["Loading..."]
    
    try:
        # Use bioio to get available scenes
        reader = BioImage(str(file_path))
        scenes = reader.scenes
        
        # Handle None or empty scenes
        if scenes is None:
            scene_dropdown.options = ["No scenes found (scenes is None)"]
            scene_dropdown.disabled = True
            update_channels(str(file_path), None)
            return
        
        # Check if scenes is iterable and has length
        try:
            num_scenes = len(scenes)
        except (TypeError, AttributeError):
            scene_dropdown.options = ["No scenes found (cannot get length)"]
            scene_dropdown.disabled = True
            update_channels(str(file_path), None)
            return
        
        if num_scenes > 0:
            if num_scenes == 1:
                # If only one scene, show it clearly
                scene_options = ["Scene 0 (only scene)"]
            else:
                # Multiple scenes
                scene_options = [f"Scene {i}" for i in range(num_scenes)]
            scene_dropdown.options = scene_options
            scene_dropdown.value = scene_options[0]
            scene_dropdown.disabled = False
            
            # Update channels for the first scene
            update_channels(str(file_path), 0)
        else:
            scene_dropdown.options = ["No scenes found"]
            scene_dropdown.disabled = True
            update_channels(str(file_path), None)
    except Exception as e:
        scene_dropdown.options = [f"Error: {str(e)}"]
        scene_dropdown.disabled = True
        channel_dropdown.options = []
        channel_dropdown.disabled = True


In [7]:
def update_channels_on_scene_change(change):
    """Update channels when scene selection changes."""
    selected_relative_path = file_dropdown.value
    selected_scene = change['new']
    
    # Check if we have valid selections
    if not selected_relative_path or selected_relative_path in ["Directory not found", "No image files found"]:
        return
    
    if not selected_scene or selected_scene in ["No scenes found", "Loading..."] or selected_scene.startswith("Error:"):
        return
    
    # Extract scene index
    try:
        scene_parts = selected_scene.split()
        if len(scene_parts) >= 2:
            scene_str = scene_parts[1]
            scene_index = int(scene_str)
            file_path = base_path / selected_relative_path
            update_channels(str(file_path), scene_index)
    except (ValueError, IndexError):
        pass


In [8]:
# Link file dropdown to scene update - this will trigger update_scenes whenever value changes
file_dropdown.observe(update_scenes, names='value')

# Link scene dropdown to channel update - this will trigger update_channels_on_scene_change when scene changes
scene_dropdown.observe(update_channels_on_scene_change, names='value')

# Initial update to populate scenes and channels for the first file
if file_list and file_list[0] not in ["Directory not found", "No image files found"]:
    update_scenes({'new': file_list[0], 'old': None, 'owner': file_dropdown, 'type': 'change'})

# Create load button
load_button = widgets.Button(
    description='Load Scene',
    disabled=False,
    button_style='info',
    tooltip='Load the selected scene from the selected file',
    icon='check',
    layout=widgets.Layout(width='200px')
)

# Output widget to display status messages
status_output = widgets.Output()

def load_scene(button):
    """Load the selected scene from the selected file into img variable."""
    with status_output:
        status_output.clear_output()
        
        # Get selected file and scene
        selected_relative_path = file_dropdown.value
        selected_scene = scene_dropdown.value
        
        # Get selected channel
        selected_channel = channel_dropdown.value
        
        # Validate selections
        if not selected_relative_path or selected_relative_path in ["Directory not found", "No image files found"]:
            print("Error: Please select a valid file")
            return
        
        # Check if selected_scene is None or invalid
        if selected_scene is None:
            print("Error: No scene selected")
            return
        
        # Check if scene is a string (should always be)
        if not isinstance(selected_scene, str):
            print(f"Error: Scene selection is not a string: {type(selected_scene)}")
            return
        
        # Check if scene is in invalid states
        invalid_scenes = ["No scenes found", "Loading..."]
        try:
            is_invalid = any(invalid in selected_scene for invalid in invalid_scenes) or selected_scene.startswith("Error:")
        except (TypeError, AttributeError):
            print(f"Error: Cannot check scene validity for: {selected_scene}")
            return
        
        if is_invalid:
            print(f"Error: Invalid scene selection: {selected_scene}")
            return
        
        # Construct full path
        file_path = base_path / selected_relative_path
        
        try:
            # Extract scene index from "Scene X" format
            # Handle both "Scene 0" and "Scene 0 (only scene)" formats
            scene_parts = selected_scene.split()
            if len(scene_parts) < 2:
                print(f"Error: Cannot parse scene from '{selected_scene}'")
                return
            
            scene_str = scene_parts[1]  # Get the number part
            scene_index = int(scene_str)
            
            # Extract channel indices if channels are selected
            options = {}
            channel_indices = []
            if selected_channel and len(selected_channel) > 0:
                # selected_channel is now a tuple/list of selected channel strings
                for channel_str in selected_channel:
                    if channel_str.startswith("Error:") or channel_str == "Loading...":
                        continue
                    try:
                        # Extract channel index from "Channel X" or "Channel X: name" format
                        channel_parts = channel_str.split()
                        if len(channel_parts) >= 2:
                            channel_idx_str = channel_parts[1].rstrip(':')
                            channel_idx = int(channel_idx_str)
                            channel_indices.append(channel_idx)
                    except (ValueError, IndexError):
                        pass
            
            # If channels are selected, handle channel selection
            if channel_indices:
                # Check if channels are contiguous
                sorted_indices = sorted(channel_indices)
                is_contiguous = all(sorted_indices[i+1] - sorted_indices[i] == 1 for i in range(len(sorted_indices)-1))
                
                if len(channel_indices) == 1:
                    # Single channel: use slice to keep dimension
                    options['C'] = slice(channel_indices[0], channel_indices[0] + 1)
                    r = load_image(str(file_path), scene_index=scene_index, options=options, squeeze=True)
                    globals()['img'] = r[0]
                    globals()["scale"] = r[1]
                elif is_contiguous:
                    # Contiguous channels: use slice
                    min_channel = min(channel_indices)
                    max_channel = max(channel_indices)
                    options['C'] = slice(min_channel, max_channel + 1)
                    r = load_image(str(file_path), scene_index=scene_index, options=options, squeeze=True)
                    globals()['img'] = r[0]
                    globals()["scale"] = r[1]
                else:
                    # Non-contiguous channels: load all channels first, then select specific ones
                    # Load without channel restriction to get all channels
                    r = load_image(str(file_path), scene_index=scene_index, squeeze=False)
                    all_channels = r[0]
                    globals()["scale"] = r[1]
                    # Find the channel dimension (typically the first dimension after spatial dims)
                    # For bioio, channels are usually in a specific dimension - try to find it
                    # Common patterns: CZYX, TCZYX, etc.
                    # We'll try to find dimension with size >= max(channel_indices) + 1
                    channel_dim = None
                    for dim_idx in range(all_channels.ndim):
                        if all_channels.shape[dim_idx] > max(channel_indices):
                            channel_dim = dim_idx
                            break
                    
                    if channel_dim is not None:
                        # Select the specific channels using advanced indexing
                        # Create index array for the channel dimension
                        indices = [slice(None)] * all_channels.ndim
                        indices[channel_dim] = sorted_indices
                        globals()['img'] = all_channels[tuple(indices)]
                    else:
                        # Fallback: assume channels are in first dimension if shape matches
                        if all_channels.shape[0] > max(channel_indices):
                            globals()['img'] = all_channels[sorted_indices]
                        else:
                            # Couldn't determine channel dimension, load with range
                            min_channel = min(channel_indices)
                            max_channel = max(channel_indices)
                            options['C'] = slice(min_channel, max_channel + 1)
                            r = load_image(str(file_path), scene_index=scene_index, options=options, squeeze=True)
                            globals()['img'] = r[0]
                            globals()["scale"] = r[1]
            else:
                # No channels selected, load all
                r = load_image(str(file_path), scene_index=scene_index)
                globals()['img'] 
                globals()["scale"]
            
            # Get img from globals for printing
            img = globals()['img']
            scale = globals()["scale"]
            
            if channel_indices:
                if len(channel_indices) == 1:
                    print(f"✓ Loaded Scene {scene_index}, Channel {channel_indices[0]} from {selected_relative_path}, scale={scale}")
                else:
                    print(f"✓ Loaded Scene {scene_index}, Channels {channel_indices} from {selected_relative_path}, scale={scale}")
            else:
                print(f"✓ Loaded Scene {scene_index} from {selected_relative_path}, scale={scale}")
            
            print(f"  Image shape: {img.shape}")
            print(f"  Image dtype: {img.dtype}")
            print(f"  Image scale: {scale}")
            
        except ValueError as e:
            print(f"Error: Invalid scene index format: {str(e)}")
            import traceback
            traceback.print_exc()
        except Exception as e:
            print(f"Error loading scene: {str(e)}")
            import traceback
            traceback.print_exc()

# Link button to load function
load_button.on_click(load_scene)


In [9]:
# Display widgets
display(widgets.VBox([
    file_dropdown, 
    scene_dropdown, 
    channel_dropdown,
    load_button,
    status_output
]))


In [10]:
dogc = DoGConfig(output_type=None, sigma_low=1.0, sigma_high=12.0)
dogc

DoGConfig(classname=None, iterator_config=ArrayIteratorConfig(classname=None, slice_def=(-2, -1)), batch_size=1, tile_shape=None, output_type=None, squeeze=True, normalize=True, sigma_low=1.0, sigma_high=12.0, mode='reflect')

In [11]:
dpn = img[1]#[40:50]
edu = img[0]#[40:50]

dog = DoG(dogc)
ddpn = dog.run(dpn)
dedu = dog.run(edu)
print (ddpn.shape)
stackview.slice(np.concatenate([ddpn, dedu], axis=-1), continuous_update=True)

2025-11-17 18:07:03,477 - INFO - Running preprocessor DoG, on stack of type uint8, True
2025-11-17 18:07:03,478 - INFO - Running DoG with config: classname=None iterator_config=ArrayIteratorConfig(classname=None, slice_def=(-2, -1)) batch_size=1 tile_shape=None output_type=None squeeze=True normalize=True sigma_low=1.0 sigma_high=12.0 mode='reflect'
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 14 concurrent workers.
2025-11-17 18:07:04,447 - INFO - Processing slice, slice.shape=(512, 512)
2025-11-17 18:07:04,447 - INFO - Processing slice, slice.shape=(512, 512)
2025-11-17 18:07:04,447 - INFO - Processing slice, slice.shape=(512, 512)
2025-11-17 18:07:04,447 - INFO - Processing slice, slice.shape=(512, 512)
2025-11-17 18:07:04,447 - INFO - Processing slice, slice.shape=(512, 512)
2025-11-17 18:07:04,447 - INFO - Processing slice, slice.shape=(512, 512)
2025-11-17 18:07:04,447 - INFO - Processing slice, slice.shape=(512, 512)
2025-11-17 18:07:04,447 - INFO - Processing slice, sl

(93, 512, 512)


In [38]:
from vistiq.seg import MicroSAMSegmenterConfig, MicroSAMSegmenter

volume_it_cfg = ArrayIteratorConfig(slice_def=(-3, -2, -1))

#th = LocalThreshold(LocalThresholdConfig(offset=0.0, block_size=51))
th = OtsuThreshold(OtsuThresholdConfig())
filters = [RangeFilter(RangeFilterConfig(attribute="area", range=(0,200000)))]
rf = RegionFilter(RegionFilterConfig(filters=filters))

lb = Labeller(LabellerConfig(iterator_config=volume_it_cfg))


In [39]:
dmask, dlabels, dresults = MicroSAMSegmenter(MicroSAMSegmenterConfig()).run(ddpn)
dmask = np.array(dmask)
dlabels = np.array(dlabels)
print (f"mask.shape={dmask.shape}, labels.shape={dlabels.shape}")

Using apple MPS device.
mask.shape=(89, 512, 512), labels.shape=(89, 512, 512)


In [40]:
emask, elabels, eresults = MicroSAMSegmenter(MicroSAMSegmenterConfig()).run(dedu)
emask = np.array(emask)
elabels = np.array(elabels)
print (f"mask.shape={emask.shape}, labels.shape={elabels.shape}")

Using apple MPS device.
mask.shape=(89, 512, 512), labels.shape=(89, 512, 512)


In [41]:
oc = OverlapConfig(method="dice", mode="outline", iterator_config=volume_it_cfg, threshold=0.1)
ovl_results = Overlap(oc).run(dlabels, elabels, ("Dpn", "EDU"))

2025-11-17 15:24:59,588 - INFO - Running Overlap with config: classname=None iterator_config=ArrayIteratorConfig(classname=None, slice_def=(-3, -2, -1)) batch_size=1 tile_shape=None output_type='list' squeeze=True output=['score', 'above_threshold'] method='dice' mode='outline' threshold=0.1


In [42]:
ovl_results[0]

[{'Dpn': 1, 'EDU': 1, 'score': 0.4668192219679634, 'above_threshold': True},
 {'Dpn': 1, 'EDU': 2, 'score': 0.0, 'above_threshold': False},
 {'Dpn': 1, 'EDU': 3, 'score': 0.0, 'above_threshold': False},
 {'Dpn': 1, 'EDU': 4, 'score': 0.0, 'above_threshold': False},
 {'Dpn': 1, 'EDU': 5, 'score': 0.0, 'above_threshold': False},
 {'Dpn': 1, 'EDU': 6, 'score': 0.0, 'above_threshold': False},
 {'Dpn': 1, 'EDU': 7, 'score': 0.0, 'above_threshold': False},
 {'Dpn': 1, 'EDU': 8, 'score': 0.0, 'above_threshold': False},
 {'Dpn': 1, 'EDU': 9, 'score': 0.0, 'above_threshold': False},
 {'Dpn': 1, 'EDU': 10, 'score': 0.0, 'above_threshold': False},
 {'Dpn': 1, 'EDU': 11, 'score': 0.0, 'above_threshold': False},
 {'Dpn': 1, 'EDU': 12, 'score': 0.0, 'above_threshold': False},
 {'Dpn': 1, 'EDU': 13, 'score': 0.0, 'above_threshold': False},
 {'Dpn': 1, 'EDU': 14, 'score': 0.0, 'above_threshold': False},
 {'Dpn': 1, 'EDU': 15, 'score': 0.0, 'above_threshold': False},
 {'Dpn': 1, 'EDU': 16, 'score': 0.0

In [43]:
print (len(ovl_results[1]["Dpn"]), len(np.unique(dlabels)), np.min(dlabels), np.max(dlabels))
ovl_results[1]["Dpn"]

178 179 0 178


,label,EDU +,max dice score
0,1,True,0.466819
1,2,True,0.311084
2,3,True,0.521339
3,4,True,0.618453
4,5,False,0.000000
...,...,...,...
173,174,True,0.101177
174,175,True,0.228562
175,176,True,0.482984
176,177,False,0.000000


In [44]:
import numpy as np
stackview.slice(np.concat([dlabels, elabels], axis=-1))

In [48]:
import napari
from napari.utils.colormaps.colormap_utils import label_colormap
viewer = napari.Viewer()
scale=(5,1,1)

In [49]:
viewer.add_image(dpn, colormap="red", blending="additive", scale=scale)
viewer.add_image(edu, colormap="green", blending="additive", scale=scale)
viewer.add_labels(dlabels, features=ovl_results[1]["Dpn"], blending="additive", scale=scale)
viewer.add_labels(elabels, features=ovl_results[1]["EDU"], blending="additive", scale=scale)

<Labels layer 'elabels' at 0x3a6079fd0>

In [50]:
dpn_df = ovl_results[1]["Dpn"]
edu_pos = dpn_df[dpn_df["EDU +"]]
edu_neg = dpn_df[~dpn_df["EDU +"]]
viewer.add_image(np.isin(dlabels, edu_pos["label"].values), name=f"edu +", blending="additive", colormap="cyan", opacity=0.5, scale=scale) 
viewer.add_image(np.isin(dlabels, edu_neg["label"].values), name=f"edu -", blending="additive", colormap="magenta", opacity=0.5, scale=scale) 

<Image layer 'edu -' at 0x3a7658200>

In [52]:
mask, labels, results = MicroSAMSegmenter(MicroSAMSegmenterConfig()).run(img[:,30:35,...])

Using apple MPS device.


AssertionError: 